In [275]:
import math
from scipy import stats
from scipy.spatial.distance import euclidean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.facecolor'] = 'white'
import matplotlib as mpl
mpl.rcParams['grid.color'] = 'lightgrey'
#needed for all graphs
colors = ['black', 'yellowgreen', 'thistle', 'slategrey', 'm']
alphas = [1, .5, 1, 1, 1]
s = 5

from scipy.sparse import csr_matrix
from src.ItemItemRecommender import ItemItemRecommender
from sklearn.model_selection import train_test_split
from src.item_item_prototype import get_ratings_data, load_movies
import warnings
warnings.filterwarnings('ignore')


# little test matrix
def ratings_matrix():
    return np.array([[4, 4, 0, 2, 2],
                     [4, 0, 0, 3, 3],
                     [4, 0, 0, 1, 1],
                     [1, 1, 1, 2, 0],
                     [2, 2, 2, 0, 0],
                     [1, 1, 1, 0, 0],
                     [5, 5, 5, 0, 0]]).astype(float)

In [276]:
# def sim_euclidean(u, v):
#     return 1 / (1 + euclidean(u, v))

# def sim_cosine(u, v):
#     costheta = np.dot(u, v)/(np.linalg.norm(u) * np.linalg.norm(v))
#     return 0.5 + 0.5 * costheta

In [277]:
# plan to impute ratings.
# input: given any two items U(i,j), and distance metric d:
# find the distance between the vectors of shared ratings
# output: matrix of distances between items

# for any two users, + distance metric
# find the distance between the vectors of shared ratings
# output: user distance matrix

# for each missing value:
# PEDRO:
# for all peter rated food
# sum(col dist: chk to item) * peter rating of j

# RILEY:
# normalize user distance matrix
# missing values = for each row
#sum across columns dist-to-peter * normed average 


In [303]:
# Pedro
utility = ratings_matrix()

n = utility.shape[0]
m = utility.shape[1]


# A matrix of distances between movies
def item_dist_mat(df):
    m = df.shape[1]
    M = df.T
    D_cos = np.zeros((m,m))
    D_euc = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            if i != j:
                z = (M[i]==0) + (M[j]==0)
                a,b = M[i][~z], M[j][~z]
                if len(a) > 0:
                    d_cos = (a.dot(b))/(np.sqrt(a.dot(a))*np.sqrt(b.dot(b)))
#                     d_euc = np.sqrt(sum((a-b)**2))
                    D_cos[i,j] = d_cos # round(np.abs(1 - d_cos),2)
#                     D_euc[i,j] = d_euc
    return D_cos

item_weights = item_dist_mat(utility)

# def user_mean_mat(df)
#     n = df.shape[0]
#     riley = df.mean(axis = 1)
#     sd = df.std(axis = 1)

# A matrix of distances between users
def user_dist_mat(df):
    m = df.shape[0]
    M = df
    D_cos = np.zeros((m,m))
    D_euc = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            if i != j:
                z = (M[i]==0) + (M[j]==0)
                a,b = M[i][~z], M[j][~z]
                if len(a) > 0:
                    d_cos = (a.dot(b))/(np.sqrt(a.dot(a))*np.sqrt(b.dot(b)))
#                     d_euc = np.sqrt(sum((a-b)**2))
                    D_cos[i,j] = d_cos
#                     D_euc[i,j] = d_euc
    return D_cos

user_weights = user_dist_mat(utility)


In [304]:
item_weights.shape # 5x5 #movies x #movies
utility

array([[4., 4., 0., 2., 2.],
       [4., 0., 0., 3., 3.],
       [4., 0., 0., 1., 1.],
       [1., 1., 1., 2., 0.],
       [2., 2., 2., 0., 0.],
       [1., 1., 1., 0., 0.],
       [5., 5., 5., 0., 0.]])

In [305]:
user_weights.shape # 7x7 #users x #users

(7, 7)

In [306]:
# X = utility
# U = user_dist_mat(utility)
# I = item_dist_mat(utility)

# for j in range(5):
#     X[0][2] = X[0].dot(I[2])/4
# X[0][2]

In [307]:
# X[0][2] = X.T[2].dot(U[0])
# X[0][2]/6


In [311]:
# lam is a parameter for weighting other peoples ratings lam closer to 1
# or your ratings lam closer to 0

def imputed_rating(utility_mat, lam = .5, var_scalar = 1.5):
    X = utility_mat.copy()
    U = user_dist_mat(utility_mat)
    I = item_dist_mat(utility_mat)
    n = utility_mat.shape[0]
    m = utility_mat.shape[1]
    for i in range(n):
        for j in range(m):
            if X[i,j] == 0:
                x = math.ceil(lam*((X[i].dot(I[j]))/(m-1))*var_scalar*(X[i][X[i]>0].var())+(1-lam)*((X.T[j].dot(U[i]))/(n-1)))
                if x > 5:
                    x = 5
                X[i,j] = x
    return X

X = imputed_rating(utility)


In [324]:
utility

array([[4., 4., 0., 2., 2.],
       [4., 0., 0., 3., 3.],
       [4., 0., 0., 1., 1.],
       [1., 1., 1., 2., 0.],
       [2., 2., 2., 0., 0.],
       [1., 1., 1., 0., 0.],
       [5., 5., 5., 0., 0.]])

In [312]:
X

array([[4., 4., 3., 2., 2.],
       [4., 2., 2., 3., 3.],
       [4., 4., 5., 1., 1.],
       [1., 1., 1., 2., 1.],
       [2., 2., 2., 1., 1.],
       [1., 1., 1., 1., 1.],
       [5., 5., 5., 1., 5.]])

In [344]:
from numpy.linalg import matrix_power

def kismet(util_mat, min_rating = 3):
    K = util_mat.copy()
    n = util_mat.shape[0]
    m = util_mat.shape[1]
    for i in range(n):
        for j in range(m):
            if K[i,j] >= 3:
                K[i,j] = 1
            else:
                K[i,j] = 0
    Ksq = K.dot(K.T)
    np.fill_diagonal(Ksq,0)
    for i in range(n):
        for j in range(m):
            if Ksq[i,j] > 1:
                Ksq[i,j] = 1
    return Ksq

k = kismet(utility)
# K = k.dot(k.T)
# np.fill_diagonal(K, 0)
# K
k2 = matrix_power(k, 2)
two_path = np.fill_diagonal(k2,0)
k, k2

(array([[0., 1., 1., 0., 0., 0., 2.],
        [1., 0., 1., 0., 0., 0., 1.],
        [1., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.]]),
 array([[0., 3., 3., 0., 0., 0., 2.],
        [2., 0., 2., 0., 0., 0., 3.],
        [2., 2., 0., 0., 0., 0., 3.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [2., 2., 2., 0., 0., 0., 0.]]))

In [330]:
utility

array([[4., 4., 0., 2., 2.],
       [4., 0., 0., 3., 3.],
       [4., 0., 0., 1., 1.],
       [1., 1., 1., 2., 0.],
       [2., 2., 2., 0., 0.],
       [1., 1., 1., 0., 0.],
       [5., 5., 5., 0., 0.]])

In [337]:
k

array([[0., 1., 1., 0., 0., 0., 2.],
       [1., 0., 1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0.]])

In [75]:
utility

array([[4., 4., 0., 2., 2.],
       [4., 0., 0., 3., 3.],
       [4., 0., 0., 1., 1.],
       [1., 1., 1., 2., 0.],
       [2., 2., 2., 0., 0.],
       [1., 1., 1., 0., 0.],
       [5., 5., 5., 0., 0.]])

In [339]:
utility - utility.mean(axis=1)

ValueError: operands could not be broadcast together with shapes (7,5) (7,) 

In [77]:
utility.std(axis=1)

array([1.49666295, 1.67332005, 1.46969385, 0.63245553, 0.9797959 ,
       0.48989795, 2.44948974])

In [80]:
a = np.ones(5)
b = np.ones(5)*5
(a.dot(b))/(np.sqrt(a.dot(a))*np.sqrt(b.dot(b)))

0.9999999999999999

In [69]:
M[1]
z
M[1][~z]
M[0][~z]

array([4., 1., 2., 1., 5.])

In [12]:




utility.T

array([[4., 4., 4., 1., 2., 1., 5.],
       [4., 0., 0., 1., 2., 1., 5.],
       [0., 0., 0., 1., 2., 1., 5.],
       [2., 3., 1., 2., 0., 0., 0.],
       [2., 3., 1., 0., 0., 0., 0.]])

In [ ]:
# Riley


In [9]:
utility

array([[4., 4., 0., 2., 2.],
       [4., 0., 0., 3., 3.],
       [4., 0., 0., 1., 1.],
       [1., 1., 1., 2., 0.],
       [2., 2., 2., 0., 0.],
       [1., 1., 1., 0., 0.],
       [5., 5., 5., 0., 0.]])

In [85]:
(utility[0][utility[0]!=0]-utility[0][utility[0]!=0].mean())+utility[0][utility[0]!=0].std()

array([2., 2., 0., 0.])

In [86]:
utility[0][utility[0]!=0].std()

1.0

In [97]:
for i in range(7):
    print((utility[i][utility[i]!=0]-.5*utility[i][utility[i]!=0].mean())+2*utility[i][utility[i]!=0].var())

[4.5 4.5 2.5 2.5]
[2.77777778 1.77777778 1.77777778]
[7. 4. 4.]
[0.75 0.75 0.75 1.75]
[1. 1. 1.]
[0.5 0.5 0.5]
[2.5 2.5 2.5]
